In [ ]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

%matplotlib widget

import torch
torch.set_float32_matmul_precision('high')

In [ ]:
# Import the relevant modules
import sousvide.synthesize.rollout_generator as rg
import sousvide.synthesize.observation_generator as og
import sousvide.instruct.train_policy as tp
import sousvide.visualize.plot_synthesize as ps
import sousvide.visualize.plot_learning as pl
import sousvide.flight.deploy_figs as df

In [ ]:
cohort = "flight_path"             # cohort name

# data_method = "data_alpha"        # method name
# data_method = "data_beta"        # method name
data_method = "eval_single"         # method name
eval_method = "eval_nominal"        # method name

scene = "mid_gate"                  # scene name

courses = [                         # course names
    # "hover",
    "line",
    # "robustness_track",
    ]   

roster = [
    # "Maverick",
    "hsDavion",
    # "hsKurita",
    # "hsLiao",
    # "hsMarik",
    # "hsSteiner"
    ]


In [ ]:
# Generate Rollouts
rg.generate_rollout_data(cohort,courses,scene,data_method)

# Review the Rollout Data
ps.plot_rollout_data(cohort)

In [ ]:
# Generate initial observation data sets
og.generate_observation_data(cohort,roster)

In [ ]:
tp.train_roster(cohort,roster,"featNet",100,regen=False)

pl.plot_losses(cohort,roster,"featNet")


In [ ]:
# Second Stage Training
tp.train_roster(cohort,["hsKurita"],"featNet",70,regen=False)

pl.plot_losses(cohort,["hsKurita"],"featNet")

In [ ]:
# Simulate in FiGS
for course in courses:
    df.deploy_roster(cohort,course,scene,eval_method,roster)
    pl.plot_deployments(cohort,course,roster,plot_show=True)